In [1]:
import sys
import os
sys.path.append(os.getcwd())
from src.taxodist import td_utils as utils
from src.taxodist import tree_parsers
from src.taxodist import td_calc
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pylab as plt
#plt.style.use("seaborn")

In [28]:
df_hd_moltb = pd.read_excel('analysis/resources/pseudo_icd10_hauptdiagnose_moltb.xlsx')
df_pancreas_hd = df_hd_moltb[df_hd_moltb['ICD-10 Hauptdiagnose'].str.contains('C25')]

df_moltb = pd.read_excel('analysis/resources/pseudo_icd10_moltb.xlsx')
df_patient_icd = df_moltb.groupby(df_moltb.PseudoPatNr.name)['ICD'].agg(list).reset_index(name='ICD')
df_pancreas_patient = df_patient_icd[df_patient_icd['PseudoPatNr'].isin(df_pancreas_hd['Pseudonym'])]

pancreas_patient_array = df_pancreas_patient.to_numpy()
pancreas_icd_sets = [set(icd_list) for patient, icd_list in pancreas_patient_array]


In [ ]:
tree = tree_parsers.getICD10GMTree(version='2021')
td = td_calc.Taxodist()
# use taxodist to calculate similarity of all patients
dist_matrix = td.calc_set_sim(pancreas_icd_sets,tree,'levels','nguyen_almubaid','bipartite_matching',normalize=False)

df_mds_coordinates = utils.getMDSMatrix(dist_matrix)


In [29]:
setnames = ['patient ' + str(count) for count, vals in enumerate(pancreas_patient_array,start=1)]
dist_matrix = pd.read_excel('analysis/generated/matrices/pancreas_pats_dist_L_NA_BIP.xlsx')
dist_matrix = dist_matrix.to_numpy()
df_mds_coordinates = utils.getMDSMatrix(dist_matrix)
df_mds_coordinates.set_axis(setnames,axis='index',inplace=True)
# utils.plotDistMatrix(df_mds_coordinates,setnames)

fig = px.scatter(x=df_mds_coordinates[0],y=df_mds_coordinates[1],hover_data=)
fig.show()

In [30]:
df_mds_coordinates

,0,1
patient 1,-117.421448,19.041668
patient 2,-55.723080,-72.776432
patient 3,-121.803837,197.696046
patient 4,21.228678,-81.043872
patient 5,-53.048662,219.983148
patient 6,283.307416,-152.876039
patient 7,188.905336,-142.025926
patient 8,-32.640105,-52.823944
patient 9,-113.382309,-27.367506
patient 10,-50.537068,-37.439500


In [ ]:
tree = tree_parsers.getICD10GMTree()
depth = tree.depth()
td = td_calc.Taxodist()

set1 = {'C00.0','C02.0','C04.0','C05.0','C06.0'}
set2 = {'D00.0','C02.0','C04.0','C05.0','C06.0'}
set3 = {'D00.0','D02.0','C04.0','C05.0','C06.0'}
set4 = {'D00.0','D02.0','D04.0','C05.0','C06.0'}
set5 = {'D00.0','D02.0','D04.0','D05.0','C06.0'}

# sets = [utils.getRandomConcepts(10,tree) for x in range(0,10)]
sets = [set1,set2,set3,set4,set5]
column_names=['Hauptdiagnose','ND1','ND2','ND3','ND4',]

setnames = ['patient ' + str(x) for x in range(1,6)]

In [ ]:
dist_matrix = td.calc_set_sim(sets,tree,'levels','nguyen_almubaid','bipartite_matching')
sim_matrix = 1.0 - dist_matrix.round(3)
df_mds_coordinates = utils.getMDSMatrix(dist_matrix)

# fig = px.sunburst(
# sets,
# names='character',
# parents='parent',
# values='value',
# )
# fig.show()


utils.plotDistMatrix(df_mds_coordinates,setnames)
dist_matrix

In [ ]:
dist_matrix = td.calc_set_sim(sets,tree,'levels','nguyen_almubaid','bipartite_matching',normalize=False)
sim_matrix = 1.0 - dist_matrix.round(3)
df_mds_coordinates = utils.getMDSMatrix(dist_matrix)

# fig = px.sunburst(
# sets,
# names='character',
# parents='parent',
# values='value',
# )
# fig.show()


utils.plotDistMatrix(df_mds_coordinates,setnames)

In [ ]:
df_sets = pd.DataFrame([['C00.0','C02.0','C04.0','C05.0','C06.0'],['D00.0','C02.0','C04.0','C05.0','C06.0'],['D00.0','D02.0','C04.0','C05.0',
            'C06.0'],['D00.0','D02.0','D04.0','C05.0','C06.0'],['D00.0','D02.0','D04.0','D05.0','C06.0']],index=setnames,columns=column_names)

df_hd = df_sets.Hauptdiagnose.unique()
nd = [df_sets[c].unique() for c in df_sets.columns[1:5]]
nd_flat = flat_list = [item for sublist in nd for item in sublist]

#ct = pd.crosstab(index=df_sets.Hauptdiagnose,columns=df_sets.columns,ag)
#ct